Visualization 8: CAR Contribution Waterfall Chart

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# This visualization requires a specific library.
# Please install it first by running: pip install waterfallcharts
try:
    import waterfall_chart
except ImportError:
    print("Error: The 'waterfallcharts' library is required for this visualization.")
    print("Please install it by running: pip install waterfallcharts")
    exit()

# --- Visualization 8: CAR Contribution Waterfall Chart ---

# Load the daily abnormal return data
try:
    df_daily_ar = pd.read_csv("./outcome/event_study_daily_ar.csv")
except FileNotFoundError:
    print("Error: 'event_study_daily_ar.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# A waterfall chart is best for analyzing a SINGLE event.
# Let's select one specific event to analyze from our data.
# You can change these parameters to analyze any event you are interested in.
asset_to_plot = 'Dogecoin'
event_date_to_plot = '2020-09-02' # Example event date
model_to_plot = 'MM_SPY'
window_size = 10

# Filter the data for this single specific event
df_plot_data = df_daily_ar[
    (df_daily_ar['Asset'] == asset_to_plot) &
    (df_daily_ar['EventDate'] == event_date_to_plot) &
    (df_daily_ar['Model'] == model_to_plot)
]

# Filter for the desired window around the event
df_plot_data = df_plot_data[df_plot_data['RelativeDay'].between(-window_size, window_size)].copy()
df_plot_data = df_plot_data.sort_values(by='RelativeDay')

if df_plot_data.empty:
    print(f"No data found for the selected event ({asset_to_plot} on {event_date_to_plot}). Cannot create waterfall chart.")
else:
    # Prepare data for the waterfall chart library
    labels = df_plot_data['RelativeDay'].astype(str).tolist()
    values = df_plot_data['AR'].tolist()
    
    # --- Plotting ---
    plt.figure(figsize=(16, 8))
    waterfall_chart.plot(
        labels,
        values,
        formatting='{:.4f}',      # Format values to 4 decimal places
        net_label='Final CAR',     # Label for the final cumulative bar
        Title=f'CAR Waterfall for {asset_to_plot} on {event_date_to_plot}\nModel: {model_to_plot}, Window: (-{window_size},+{window_size})'
    )
    
    # --- Chart Enhancement ---
    plt.ylabel('Abnormal Return (AR) Contribution')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.show()

Visualization 9: Model Fit Diagnostic (Scatter Plot)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Visualization 9: Model Fit Diagnostic (Scatter Plot) ---

# Load the estimation period data
try:
    df_estimation = pd.read_csv("./outcome/event_study_estimation.csv")
except FileNotFoundError:
    print("Error: 'event_study_estimation.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# Like the waterfall chart, this diagnostic is for a SINGLE event's estimation period.
# Let's select one specific event's estimation to analyze.
asset_to_plot = 'Bitcoin'
event_date_to_plot = '2020-09-02' # Example event date
model_to_plot = 'MM_SPY'

# Filter the data for this single specific estimation period
df_plot_data = df_estimation[
    (df_estimation['Asset'] == asset_to_plot) &
    (df_estimation['EventDate'] == event_date_to_plot) &
    (df_estimation['Model'] == model_to_plot)
]

# Determine the benchmark column based on the model
benchmark_col = 'SPY' # The column name was standardized to lowercase

if df_plot_data.empty:
     print(f"No estimation data found for the selected event ({asset_to_plot} on {event_date_to_plot}).")
else:
    # --- Plotting ---
    plt.figure(figsize=(10, 10))
    
    # Use seaborn's regplot to create a scatter plot and automatically fit an OLS regression line
    sns.regplot(
        data=df_plot_data,
        x=benchmark_col,
        y='AssetReturn',
        line_kws={"color": "red", "lw": 2}, # Customize the regression line
        scatter_kws={"alpha": 0.5, "s": 50}  # Customize the scatter points
    )
    
    # --- Chart Enhancement ---
    plt.title(f'Model Fit Diagnostic for Estimation Period\nEvent: {asset_to_plot} on {event_date_to_plot}, Model: {model_to_plot}', fontsize=16)
    plt.xlabel(f'Benchmark Daily Return ({benchmark_col.upper()})', fontsize=12)
    plt.ylabel(f'Asset Daily Return ({asset_to_plot})', fontsize=12)
    plt.grid(True, linestyle='--', linewidth=0.5)
    
    plt.show()